<a href="https://colab.research.google.com/github/NvdSuni/Thesis-code-complete/blob/main/FFN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install optuna
!pip install imbalanced-learn

import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers, models
import optuna
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
from imblearn.over_sampling import SMOTE
import optuna
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import __version__ as sklearn_version
from tensorflow.keras import optimizers

import imblearn
imblearn_version = imblearn.__version__

print(f"optuna: {optuna.__version__}")
print(f"imbalanced-learn: {imblearn_version}")
print(f"matplotlib: {plt.matplotlib.__version__}")
print(f"numpy: {np.__version__}")
print(f"PIL (Pillow): {Image.__version__}")
print(f"tensorflow: {tf.__version__}")
print(f"scikit-learn.utils: {sklearn_version}")


In [ ]:
X_train_combined = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_combined.npy")
X_val_combined = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_combined.npy")
y_train_combined = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_combined.npy")
y_val_combined = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_combined.npy")

#Untuned model

In [ ]:
y_train_combined = tf.keras.utils.to_categorical(y_train_combined, num_classes=7)
y_val_combined = tf.keras.utils.to_categorical(y_val_combined, num_classes=7)
np.random.seed(42)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train_combined.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_combined, y_train_combined, epochs=30, validation_data=(X_val_combined, y_val_combined))

y_pred = model.predict(X_val_combined)
y_true = y_val_combined.argmax(axis=1)
y_pred_classes = y_pred.argmax(axis=1)

roc_auc = roc_auc_score(y_val_combined, y_pred, multi_class='ovr')
print(f"ROC-AUC: {roc_auc}")

print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

model.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN")

#Tuned model

In [ ]:
def create_model(trial):
    model = models.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(X_train_combined.shape[1],)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(7, activation='softmax'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping_auc = EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

    history = model.fit(
        X_train_combined,
        y_train_combined,
        epochs=30,
        validation_data=(X_val_combined, y_val_combined),
        verbose=1,
        callbacks=[early_stopping_auc]
    )

    y_pred_proba = model.predict(X_val_combined)
    roc_auc = roc_auc_score(y_val_combined, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')

    return roc_auc


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)


best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")


best_model = create_model(optuna.trial.FixedTrial(best_params))
history = best_model.fit(X_train_combined, y_train_combined, epochs=30, validation_data=(X_val_combined, y_val_combined))

y_pred = best_model.predict(X_val_combined)
y_true = y_val_combined.argmax(axis=1)
y_pred_classes = y_pred.argmax(axis=1)

final_roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
print(f'Final Model ROC-AUC: {final_roc_auc}')

print("Final Model Classification Report:")
print(classification_report(y_true, y_pred_classes))

best_model.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_optuna_auc")


#Tuned model + Class imbalance treatment

In [ ]:
smote = SMOTE(random_state=42)

X_train_combined_balanced, y_train_combined_balanced = smote.fit_resample(X_train_combined, y_train_combined)

In [ ]:
def create_model(trial):
    model = models.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(X_train_combined_balanced.shape[1],)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(7, activation='softmax'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping_auc = EarlyStopping(monitor='val_auc', patience=5, mode = 'max', restore_best_weights=True)

    history = model.fit(
        X_train_combined_balanced,
        y_train_combined_balanced,
        epochs=30,
        validation_data=(X_val_combined, y_val_combined),
        verbose=1,
        callbacks=[early_stopping_auc]
    )

    y_pred_proba = model.predict(X_val_combined)
    roc_auc = roc_auc_score(y_val_combined, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')


    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)


best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_model_balanced = create_model(optuna.trial.FixedTrial(best_params))
history = best_model_balanced.fit(X_train_combined_balanced, y_train_combined_balanced, epochs=30, validation_data=(X_val_combined, y_val_combined))


y_pred_balanced = best_model_balanced.predict(X_val_combined)
y_true_balanced = y_val_combined.argmax(axis=1)
y_pred_classes_balanced = y_pred_balanced.argmax(axis=1)


final_roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
print(f'Final Model ROC-AUC: {final_roc_auc}')


print("Final Model Classification Report:")
print(classification_report(y_true_balanced, y_pred_classes_balanced))

best_model_balanced.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_smote")


#Xray specific

In [ ]:
X_train_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_Xray_reduced.npy")
X_val_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_Xray_reduced.npy")
y_train_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/train_labels_complete_Xray.npy")
y_val_Xray = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/val_labels_complete_Xray.npy")

In [ ]:
from tensorflow.keras.utils import to_categorical

X_train_shuffled_Xray, y_train_shuffled_Xray = shuffle(X_train_Xray, y_train_Xray, random_state=42)
X_val_shuffled_Xray, y_val_shuffled_Xray = shuffle(X_val_Xray, y_val_Xray, random_state=42)

print(X_train_shuffled_Xray.shape, y_train_shuffled_Xray.shape, X_val_shuffled_Xray.shape, y_val_shuffled_Xray.shape)

print(y_train_shuffled_Xray)

#FFN Untuned: Xray

In [ ]:
import numpy as np
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers

model_Xray = keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model_Xray.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model_Xray.fit(X_train_shuffled_Xray, y_train_shuffled_Xray, epochs=30, validation_data=(X_val_shuffled_Xray, y_val_shuffled_Xray))

y_val_pred = model_Xray.predict(X_val_shuffled_Xray)

y_val_pred_labels = np.argmax(y_val_pred, axis=1)

print("Classification Report:")
print(classification_report(np.argmax(y_val_shuffled_Xray, axis=1), y_val_pred_labels))

roc_auc_avg_Xray = roc_auc_score(y_val_shuffled_Xray, y_val_pred, average='macro')
print(f"Average ROC-AUC Score: {roc_auc_avg_Xray}")

model_Xray.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_Xray")

#FFN Tuned: Xray

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from sklearn.metrics import roc_auc_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping

def create_model(trial):
    model = models.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(64,)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(5, activation='softmax'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping = EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

    history = model.fit(
        X_train_shuffled_Xray,
        y_train_shuffled_Xray,
        epochs=30,
        validation_data=(X_val_shuffled_Xray, y_val_shuffled_Xray),
        verbose=1,
        callbacks=[early_stopping]
    )

    y_pred_proba = model.predict(X_val_shuffled_Xray)
    roc_auc = roc_auc_score(y_val_shuffled_Xray, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')

    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_model_Xray = create_model(optuna.trial.FixedTrial(best_params))
history = best_model_Xray.fit(X_train_shuffled_Xray, y_train_shuffled_Xray, epochs=30, validation_data=(X_val_shuffled_Xray, y_val_shuffled_Xray))

y_pred = best_model_Xray.predict(X_val_shuffled_Xray)
y_true = np.argmax(y_val_shuffled_Xray, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

final_roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
print(f'Final Model ROC-AUC: {final_roc_auc}')

print("Final Model Classification Report:")
print(classification_report(y_true, y_pred_classes))

best_model_Xray.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_Xray")


#FFN Tuned + SMOTE - Xray

In [ ]:
smote = SMOTE(random_state=42)

X_train_combined_balanced_Xray, y_train_combined_balanced_Xray = smote.fit_resample(X_train_shuffled_Xray, y_train_shuffled_Xray)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def create_model(trial):
    model = models.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(64,)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(5, activation='softmax'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping = EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

    model_checkpoint = ModelCheckpoint('best_model_FFN_Xray_specific_tuned_class_imbalance.h5', monitor='val_auc', mode='max', save_best_only=True)


    history = model.fit(
        X_train_combined_balanced_Xray,
        y_train_combined_balanced_Xray,
        epochs=30,
        validation_data=(X_val_shuffled_Xray, y_val_shuffled_Xray),
        verbose=1,
        callbacks=[early_stopping]
    )

    y_pred_proba = model.predict(X_val_shuffled_Xray)
    roc_auc = roc_auc_score(y_val_shuffled_Xray, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')

    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_model_smote_Xray = create_model(optuna.trial.FixedTrial(best_params))
history = best_model_smote_Xray.fit(X_train_combined_balanced_Xray, y_train_combined_balanced_Xray, epochs=30, validation_data=(X_val_shuffled_Xray, y_val_shuffled_Xray))

y_pred = best_model_smote_Xray.predict(X_val_shuffled_Xray)
y_true = np.argmax(y_val_shuffled_Xray, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

final_roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
print(f'Final Model ROC-AUC: {final_roc_auc}')

print("Final Model Classification Report:")
print(classification_report(y_true, y_pred_classes))

best_model_smote_Xray.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_smote_Xray")


#MRI Specific

In [ ]:
X_train_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_train_MRNet_reduced.npy")
X_val_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/X_val_MRNet_reduced.npy")
y_train_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_MRNet.npy")
y_val_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_MRNet.npy")

In [ ]:
model_MRI = keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_MRI.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='auc')])


history = model_MRI.fit(X_train_MRNet, y_train_MRNet, epochs=30, validation_data=(X_val_MRNet, y_val_MRNet))


y_val_pred = model_MRI.predict(X_val_MRNet)


y_val_pred_labels = (y_val_pred > 0.5).astype(int)

print("Classification Report:")
print(classification_report(y_val_MRNet, y_val_pred_labels))

roc_auc = roc_auc_score(y_val_MRNet, y_val_pred)
print(f"ROC-AUC Score: {roc_auc}")

model_MRI.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_MRI")

#MRI Tuned

In [ ]:
def create_model(trial):
    model = keras.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(64,)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(1, activation='sigmoid'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

    model_checkpoint = ModelCheckpoint('best_model_FFN_Xray_specific_tuned.h5', monitor='val_auc', mode='max', save_best_only=True)

    history = model.fit(
        X_train_MRNet,
        y_train_MRNet,
        epochs=30,
        validation_data=(X_val_MRNet, y_val_MRNet),
        verbose=1,
        callbacks=[early_stopping]
    )

    y_pred_proba = model.predict(X_val_MRNet)
    roc_auc = roc_auc_score(y_val_MRNet, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')

    return roc_auc


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)


best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_model_MRI = create_model(optuna.trial.FixedTrial(best_params))
history = best_model_MRI.fit(X_train_MRNet, y_train_MRNet, epochs=30, validation_data=(X_val_MRNet, y_val_MRNet))

y_pred = best_model_MRI.predict(X_val_MRNet)
y_pred_classes = (y_pred > 0.5).astype(int)
y_true = y_val_MRNet

final_roc_auc = roc_auc_score(y_true, y_pred)
print(f'Final Model ROC-AUC: {final_roc_auc}')

print("Final Model Classification Report:")
print(classification_report(y_true, y_pred_classes))

best_model_MRI.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_MRI")

#MRI Tuned + SMOTE

In [ ]:
smote = SMOTE(random_state=42)

X_train_combined_balanced_MRNet, y_train_combined_balanced_MRNet = smote.fit_resample(X_train_MRNet, y_train_MRNet)

In [ ]:
def create_model(trial):
    model = keras.Sequential()
    model.add(layers.Dense(trial.suggest_int('units1', 32, 256), activation='relu', input_shape=(64,)))

    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 8)

    for i in range(num_dense_layers):
        model.add(layers.Dense(trial.suggest_int(f'units_dense_{i}', 16, 128), activation=trial.suggest_categorical(f'activation_dense_{i}', ['relu', 'sigmoid', 'tanh'])))
        model.add(layers.Dropout(trial.suggest_float(f'dropout_dense_{i}', 0.0, 0.5)))

    model.add(layers.Dense(1, activation='sigmoid'))

    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        optimizer = optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

def objective(trial):
    model = create_model(trial)

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True)

    model_checkpoint = ModelCheckpoint('best_model_FFN_Xray_specific_tuned_class_imbalance.h5', monitor='val_auc', mode='max', save_best_only=True)

    history = model.fit(
        X_train_combined_balanced_MRNet,
        y_train_combined_balanced_MRNet,
        epochs=30,
        validation_data=(X_val_MRNet, y_val_MRNet),
        verbose=1,
        callbacks=[early_stopping]
    )

    y_pred_proba = model.predict(X_val_MRNet)
    roc_auc = roc_auc_score(y_val_MRNet, y_pred_proba)

    print(f'Trial {trial.number} ROC-AUC: {roc_auc}')

    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_model_smote_MRI = create_model(optuna.trial.FixedTrial(best_params))
history = best_model_smote_MRI.fit(X_train_combined_balanced_MRNet, y_train_combined_balanced_MRNet, epochs=30, validation_data=(X_val_MRNet, y_val_MRNet))

y_pred = best_model_smote_MRI.predict(X_val_MRNet)
y_pred_classes = (y_pred > 0.5).astype(int)
y_true = y_val_MRNet

final_roc_auc = roc_auc_score(y_true, y_pred)
print(f'Final Model ROC-AUC: {final_roc_auc}')

print("Final Model Classification Report:")
print(classification_report(y_true, y_pred_classes))

best_model_smote_MRI.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/Models/FFN/FFN_tuned_smote_MRI")
